In [2]:
import numpy as np
from pathlib import Path
import re
import itertools


In [3]:
with Path("../05.in").open() as f:
    data = f.read().splitlines()


In [4]:
testdata = """\
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".splitlines()


In [5]:
class RangeDict(dict):
    """A dictionary that returns the value of the closest smaller key if the key is missing."""

    def __getitem__(self, key):
        if key in self:
            source, dest, range = key, *super().__getitem__(key)
        else:
            source, dest, range = self.__missing__(key)

        if source + range < key:
            # If the key is not in the range, it is a 1:1 mapping
            return key
        else:
            diff = key - source
            return dest + diff

    def __missing__(self, key):
        # Get the closest smaller key instead
        difference = sorted([key - k for k in self.keys() if key-k >= 0])
        if len(difference) == 0:
            # Needed in case the key is smaller than the smallest key
            return (key, key, 1)
        new_key = key - difference[0]
        return new_key, *super().__getitem__(new_key)


# initialize with {source: (dest, range)}
# infer with any source key
# returns 1:1 mapping if key is not in range
# otherwise returns the result of the mapping
RangeDict({98: (50, 2), 50: (52, 48)})[79] == 81


True

### Mapping Definition


| Destination | Source | Length/Range |
| ----------- | ------ | ------------ |
| 50          | 98     | 2            |
| 52          | 50     | 48           |

**Any source numbers that aren't mapped correspond to the same destination number.**


In [6]:
seeds: list = []

seed_to_soil:               dict[int, tuple[int, int]] = RangeDict()
soil_to_fertilizer:         dict[int, tuple[int, int]] = RangeDict()
fertilizer_to_water:        dict[int, tuple[int, int]] = RangeDict()
water_to_light:             dict[int, tuple[int, int]] = RangeDict()
light_to_temperature:       dict[int, tuple[int, int]] = RangeDict()
temperature_to_humidity:    dict[int, tuple[int, int]] = RangeDict()
humidity_to_location:       dict[int, tuple[int, int]] = RangeDict()


In [7]:
section_map = {
    "seed-to-soil map": seed_to_soil,
    "soil-to-fertilizer map": soil_to_fertilizer,
    "fertilizer-to-water map": fertilizer_to_water,
    "water-to-light map": water_to_light,
    "light-to-temperature map": light_to_temperature,
    "temperature-to-humidity map": temperature_to_humidity,
    "humidity-to-location map": humidity_to_location,
}


In [8]:
seeds = [int(x) for x in re.findall(r"\d+", data[0])]


In [9]:
current_section = None
for line in data[1:]:
    if ":" in line:
        current_section = section_map[line.split(":")[0]]
        continue
    if len(line) == 0:
        continue

    dest, source, length = [int(x) for x in line.split()]
    current_section[source] = tuple([dest, length])


In [10]:
def seeds_to_locations(seeds: list) -> list:
    locations = []

    for seed in seeds:
        soil = seed_to_soil[seed]
        fertilizer = soil_to_fertilizer[soil]
        water = fertilizer_to_water[fertilizer]
        light = water_to_light[water]
        temperature = light_to_temperature[light]
        humidity = temperature_to_humidity[temperature]
        location = humidity_to_location[humidity]
        locations.append(location)
        # print(f"Seed {seed} grows in location {location}")
        # print(f"({seed}->{soil}->{fertilizer}->{water}->{light}->{temperature}->{humidity}->{location})\n")

    return locations



## Part I

In [11]:
locations = seeds_to_locations(seeds)
min_location = min(locations)
print(f"\nMinimum location is {min_location}")



Minimum location is 51580674


## Part II

In [12]:

seed_ranges = list(zip(seeds[::2], seeds[1::2]))


In [13]:

number_of_checks = sum([x[1] for x in seed_ranges])
max_location = max([start+range for start, range in humidity_to_location.values()])
min_known_location = min(locations[::2])

print(f"Naively (forward), we would need to check {number_of_checks:,} seeds")
print(f"Naively (backward), we would need to check {max_location:,} locations")
print(f"Lowest known location is {min_known_location:,}")


Naively (forward), we would need to check 1,680,883,088 seeds
Naively (backward), we would need to check 4,294,967,296 locations
Lowest known location is 433,589,209


In [14]:
location_to_humidity    = RangeDict({dest: (source, length) for (source, (dest, length)) in humidity_to_location.items()})
humidity_to_temperature = RangeDict({dest: (source, length) for (source, (dest, length)) in temperature_to_humidity.items()})
temperature_to_light    = RangeDict({dest: (source, length) for (source, (dest, length)) in light_to_temperature.items()})
light_to_water          = RangeDict({dest: (source, length) for (source, (dest, length)) in water_to_light.items()})
water_to_fertilizer     = RangeDict({dest: (source, length) for (source, (dest, length)) in fertilizer_to_water.items()})
fertilizer_to_soil      = RangeDict({dest: (source, length) for (source, (dest, length)) in soil_to_fertilizer.items()})
soil_to_seed            = RangeDict({dest: (source, length) for (source, (dest, length)) in seed_to_soil.items()})


In [15]:
def location_to_seed(location: int) -> int:
    humidity = location_to_humidity[location]
    temperature = humidity_to_temperature[humidity]
    light = temperature_to_light[temperature]
    water = light_to_water[light]
    fertilizer = water_to_fertilizer[water]
    soil = fertilizer_to_soil[fertilizer]
    seed = soil_to_seed[soil]
    return seed


In [34]:
def in_range(x):
    return any([x in range(start, start+length) for (start, length) in seed_ranges])


def get_seed_locations(locations: list) -> tuple[int, int, int]:
    print(f"Checking locations starting at {locations[0]}")
    for location in locations:
        # if location % 100_000 == 0:
            # print(f"Checking location {location}")
        humidity = location_to_humidity[location]
        temperature = humidity_to_temperature[humidity]
        light = temperature_to_light[temperature]
        water = light_to_water[light]
        fertilizer = water_to_fertilizer[water]
        soil = fertilizer_to_soil[fertilizer]
        seed = soil_to_seed[soil]

        if in_range(seed):
            print(f"Seed {seed} grows in location {location}")
            return locations[0], seed, location

# takes too long - stopped at around 52.000.000 (~20min)
# with roughly 1.21% locations checked (although we only need the first match)
# get_seed_locations(range(min_known_location))


In [35]:
from concurrent.futures import ProcessPoolExecutor, as_completed


def get_seed_locations_multiproc(max_location, num_worker=16) -> list:

    stepsize = max_location // num_worker
    starting_indices = [stepsize * i for i in range(num_worker)]
    ranges = [range(start, start+stepsize) for start in starting_indices]

    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        futures = [executor.submit(get_seed_locations, r) for r in ranges]

        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                start, seed, loc = result
                if np.min(starting_indices) == start:
                    # Current is the lowest match, abort the rest.
                    executor.shutdown(wait=False)
                    # Today I learned:
                    #   - shutdown(wait=False) does not kill the running processes
                    #   - cancel() does not kill the running processes either
                    #   - in fact there is no easy way to kill the running processes,
                    #     only to cancel them before they start.
                    return start, seed, loc
                else:
                    starting_indices.remove(start)


In [36]:
min_known_locations = [
    min_known_location,  # This is an already known minimum
    100_267_484,  # This one was found by luck (more or less) and reduced the search space immensely
]


In [37]:
get_seed_locations_multiproc(min_known_locations[-1], num_worker=16)


Checking locations starting at 0Checking locations starting at 25066868Checking locations starting at 37600302Checking locations starting at 31333585Checking locations starting at 18800151Checking locations starting at 12533434Checking locations starting at 50133736Checking locations starting at 6266717Checking locations starting at 56400453Checking locations starting at 43867019Checking locations starting at 62667170
Checking locations starting at 68933887Checking locations starting at 75200604


Checking locations starting at 94000755Checking locations starting at 87734038Checking locations starting at 81467321













Seed 1055427336 grows in location 99751240


In [38]:
99_751_240


99751240